## Download files

In [1]:
# Read json source
import json
from os import path

SourceData = json.load(open(path.abspath('./data/data.json')))

In [37]:
import requests, os

OUTPUT_PATH = path.abspath('./data/generated/input')

def saveFile(type, candidat, filename, content):
    final_dir = path.join(OUTPUT_PATH, type, candidat)
    if not os.path.exists(final_dir):
        os.makedirs(final_dir)
    with open(path.join(final_dir, filename), 'wb') as f:
        f.write(content)

def downloadPdf(candidat, current_url):
    resp = requests.get(current_url)
    if resp.status_code != 200:
        print("\ERROR ({}) for {}".format(resp.status_code, current_url))
    else:
        filename = current_url.split("/")[-1]
        saveFile('pdf', candidat, filename, resp.content)
            
def downloadHtml(candidat, current_url):
    resp = requests.get(current_url)
    if resp.status_code != 200:
        print("\ERROR ({}) for {}".format(resp.status_code, current_url))
    else:
        filename = (current_url[:-1] if (current_url.endswith("/")) else current_url).split("/")[-1]
        saveFile('pdf', candidat, filename+".html", resp.content)

In [38]:
# Download files
for candidat in SourceData:
    for current_url in SourceData[candidat]:
        if current_url.endswith(".pdf"):
            downloadPdf(candidat, current_url)
        else:
            downloadHtml(candidat, current_url)

\ERROR (409) for https://melenchon2022.fr/wp-content/uploads/2022/02/Livret_UP_06_MIGRATION_web-1.pdf


TypeError: downloadHtml() missing 1 required positional argument: 'current_url'